In [ ]:
!pip install ipython-autotime
%load_ext autotime
import os
import pandas as pd
import numpy as np
import json
import re
import datetime
import datetime as dt
from tqdm import tqdm

In [ ]:
# if PART_SIZE: # Load only {PART_SIZE}_MB of data file
#     data_file_part = r'Part_%d_MB.csv' % PART_SIZE
#     if not os.path.exists(data_file_part):
#         lines = open(data_file).readlines(PART_SIZE*1024*1024)[:-1]
#         open(data_file_part, 'wt').writelines(lines)
#     df = pd.read_csv(data_file_part)
# else:
#     df = pd.read_csv(data_file)

time: 1.98 ms (started: 2022-07-20 13:59:23 +00:00)


In [ ]:
# Setting up lists for importing the data.
path_data = '/content/drive/MyDrive/Tai-Sana_exchangeDrive/pr-1_1/dataset/Mar22'
list_of_csv = os.listdir(path_data)
list_of_csv

['Output1.csv',
 'Output2.csv',
 'Output3.csv',
 'Output4.csv',
 'Output5.csv',
 'Output6.csv',
 'Output7.csv',
 'Output8.csv',
 'Output9.csv',
 'Output11.csv',
 'Output10.csv',
 'Output13.csv',
 'Output12.csv',
 'Output14.csv',
 'Output15.csv',
 'Output17.csv',
 'Output16.csv',
 'Output18.csv',
 'Output20.csv',
 'Output19.csv']

time: 5.91 ms (started: 2022-08-02 04:44:34 +00:00)


In [4]:
df_newACs = pd.read_csv('/content/drive/MyDrive/Tai-Sana_exchangeDrive/pr-1_1/Result_TAI/update_newACs_0108.csv', index_col=0)
df_newACs

,SourceId
0,c44b2908-c437-4edf-a52f-4562219065b1
1,4baea418-26f6-40f4-8ccb-0929e02d1fce
2,90cfb0cc-cdfc-4e17-866e-7d722bc42dd9
3,4d1f8059-6964-43a2-8684-0a58daa3f741
4,ba05239a-3a5f-4130-985a-1ded0be89223
...,...
10104,c9728cd8-1676-498e-8f41-5d55b7b72106
10105,2c502fc0-6d43-43e9-91e1-2e2e4580622c
10106,0c931903-1e18-4f0f-bc6f-10c22c317a9c
10107,51607529-f208-49b8-b409-860eb898dc5d


time: 766 ms (started: 2022-08-17 18:24:45 +00:00)


In [ ]:
for i in tqdm(list_of_csv):
  if i.endswith('.csv'):
    # Read each .csv file in folder
    k = pd.read_csv(path_data + '/' + i, sep='delimiter', engine ='python')
    k[["Column_1","MessageId","EventUtcTime","Timestamp","Cmd","SourceId","Payload","PayloadJson"]] = k['Column_1;MessageId;EventUtcTime;Timestamp;Cmd;SourceId;Payload;PayloadJson'].str.split(";", expand=True)
    k.drop(columns=['Column_1;MessageId;EventUtcTime;Timestamp;Cmd;SourceId;Payload;PayloadJson','Column_1'], inplace=True)
    print('Len of orginal file:',len(k))

    # BASED ON ATTRIBUTES:
    # Replace the blank values with NaN  
    k = k.replace(r'^\s*$', np.NaN, regex=True)
    k = k.replace(r'null', np.NaN, regex=True)
    # Drop row has NaN in SourceId and Cmd columns
    k.dropna(subset=['SourceId', 'Cmd'],inplace = True)
    print('Len of k after check SourceID and Cmd:', len(k))

    # Remove OLD ACs
    k = k.merge(df_newACs, on=["SourceId"])
    k['EventUtcTime'] = pd.to_datetime(k['EventUtcTime'])
    k.sort_values(by=['EventUtcTime'], inplace = True, ignore_index = True)
    print('Len of k after remove Old ACs:',len(k))

    # Replace mising json strings by {}
    k[['Payload','PayloadJson']] = k[['Payload','PayloadJson']].fillna('{}')
    k.reset_index(inplace=True, drop =True)
    
    # Convert type of Payload & PayloadJson into string
    k['Payload'] = k['Payload'].astype(str)
    k['PayloadJson'] = k['PayloadJson'].astype(str)
    
    # Expand json column to dataframe
    json1 = pd.json_normalize(k['Payload'].apply(json.loads))
    json2 = pd.json_normalize(k['PayloadJson'].apply(json.loads))

    # json1 = json1.replace(r'^\s*$', np.NaN, regex=True)
    # json2 = json2.replace(r'^\s*$', np.NaN, regex=True)

    # Join json1 and join2 to k 
    k = k.join(json1).join(json2)

    # Add new columns
    k['PayloadLen'] = k['Payload'].apply(json.loads).apply(len)
    k['PayloadJsonLen'] = k['PayloadJson'].apply(json.loads).apply(len)
    # k.info()

    # Check wrong format Timestamp
    k = k[k.Timestamp.str.match(r'^00[a-zA-Z0-9]{8}|0000000$', na = False)]

    # Check wrong value Timestamp
    k['Timestamp'] = k['Timestamp'].apply(int, base=16)
    k['Timestamp'] = k['Timestamp'].replace(0, np.NaN)
    k['Timestamp'] = pd.to_datetime(k['Timestamp'], unit='s')
    # If the timestamp value is equal to 0 (NaN), so we account the evtUTCTime as timeStamp.
    k['Timestamp'] = k['Timestamp'].fillna(k['EventUtcTime'])
    k = k[(k['Timestamp'] >= '2022-03-01 00:00:00.000') & (k['Timestamp'] <= '2022-03-31 23:59:59.999')] #filter January
    print('Len of k after check timestampt:', len(k))

    # BASED ON THE CONTENT OF JSON PAYLOAD

    # If Cmd = CMD_FCU_FROM_AC. We are neglecting packets with: Empty JSON payload (PayloadJson columns). The value of Payload columns is no meaningful.
    k = k[(k.PayloadJsonLen != 0) | (k.PayloadLen != 0) | (k.Cmd != 'CMD_FCU_FROM_AC')]

    # Check if the ac_outdoor_temp = 126 or 127, if True, remove the row having that parameter.
    k = k[(k.ac_outdoor_temp != (126 or 127)) | (k.Cmd != 'CMD_FCU_FROM_AC')]
    print('Len of k after check  the ac_outdoor_temp:', len(k))

    # Check ac_scheduler_status is exist or not, if True, remove the rows having 'ac_scheduler_status'
    k = k[(k['ac_scheduler_status'].isnull()) | (k.Cmd != 'CMD_FCU_FROM_AC')] # Keeping the row that has 'ac_scheduler_status = NaN' & 'CMD_FCU_FROM_AC'
    print('Len of k after check ac_scheduler_status is exist or not:', len(k))

    # Check 'ac_uiled_status' is exist or not, if True, remove the rows having 'ac_uiled_status'
    k = k[(k['ac_uiled_status'].isnull()) | (k.Cmd != 'CMD_FCU_FROM_AC')]
    print('Len of k after check ac_uiled_status is exist or not:', len(k))

    # We sometimes have two payloads (column 7 and column 8): 1st payload has the actual HEARTBEAT parameters
    # while the 2nd one had extra ac_indoor_temp and ac_outdoor_temp. We are neglecting packets with: Empty 1st payload: missing HEARTBEAT parameters
    k = k[(k.PayloadLen != 0) | (k.Cmd != 'CMD_HEARTBEAT')]

    # If CMD in {CMD_ENERGY_TODAY, CMD_ENERGY_THIS_WEEK, CMD_ENERGY_THIS_MONTH, CMD_ENERGY_THIS_YEAR}.
    # We are neglecting packets with: Empty JSON payload ( both Payload columns and PayloadJson columns).

    # If cmd = CMD_ENERGY_TODAY
    k = k[(k.PayloadJsonLen != 0) | (k.PayloadLen != 0) | (k.Cmd != 'CMD_ENERGY_TODAY')]

    #if Cmd = CMD_ENERGY_THIS_WEEK
    k = k[(k.PayloadJsonLen != 0) | (k.PayloadLen != 0) | (k.Cmd != 'CMD_ENERGY_THIS_WEEK')]

    # if Cmd = CMD_ENERGY_THIS_MONTH
    k = k[(k.PayloadJsonLen != 0) | (k.PayloadLen != 0) | (k.Cmd != 'CMD_ENERGY_THIS_MONTH')]

    #  if Cmd = CMD_ENERGY_THIS_YEAR
    k = k[(k.PayloadJsonLen != 0) | (k.PayloadLen != 0) | (k.Cmd != 'CMD_ENERGY_THIS_YEAR')]
    print('len of k after check CMD:',len(k))
    # k.info()

    k.drop(columns=['PayloadJsonLen', 'PayloadLen','Payload', 'PayloadJson', 'ac_scheduler_status','ac_uiled_status','data','GMT_MM', 'GMT_HH'], inplace = True)
    # df.info()

    # rename and drop a few columns
    k.rename(columns = {'iTemp':'HB-iTemp','oTemp':'HB-oTemp','fcuTcTemp':'HB-fcuTcTemp','fcuTcjTemp':'HB-fcuTcjTemp',
                          'fcuFanRpm':'HB-fcuFanRpm','cduTdTemp':'HB-cduTdTemp','cduTsTemp':'HB-cduTsTemp','cduTeTemp':'HB-cduTeTemp',
                          'cduCompHz':'HB-cduCompHz','cduFanRpm':'HB-cduFanRpm','cduPmvPulse':'HB-cduPmvPulse','cduIac':'HB-cduIac',
                          'ac_on_standby':'FCU-ac_on_standby',
                          'ac_mode':'FCU-ac_mode','ac_temp':'FCU-ac_temp','ac_airflow':'FCU-ac_airflow','ac_airswing':'FCU-ac_airswing',
                          'ac_power_sel':'FCU-ac_power_sel','ac_merit_mode':'FCU-ac_merit_mode','ac_air_purifier':'FCU-ac_air_purifier',
                          'error_code':'FCU-error_code','ac_timer_type':'FCU-ac_timer_type','ac_self_cleaning':'FCU-ac_self_cleaning'}, inplace = True)

    k.drop(columns=['fcu_tc_temp','fcu_fan_rpm','cdu_td_temp','cdu_ts_temp','cdu_te_temp','cdu_comp_hz',
                      'cdu_fan_rpm','cdu_pmv_pulse','cdu_iac','fcu_tcj_temp','ac_timer_mm','ac_timer_hh'], inplace = True)
    
    k[['ac_indoor_temp','ac_outdoor_temp']] = k[['ac_indoor_temp','ac_outdoor_temp']].astype(str)

    # This block code, I will create: 
    # EM-fcuTime, EW-fcuTime, EY-fcuTime, ET-fcuTime, EM-energy, EW-energy, EY-energy, ET-energy,
    # FCU-ac_indoor_temp, HB-ac_indoor_temp, FCU-ac_outdoor_temp, HB-ac_outdoor_temp

    result1 = k[["Cmd"]].join(pd.get_dummies(k["Cmd"]).mul(k["fcuTime"], axis=0))
    result2 = k[["Cmd"]].join(pd.get_dummies(k["Cmd"]).mul(k["energy"], axis=0))
    result3 = k[["Cmd"]].join(pd.get_dummies(k["Cmd"]).mul(k["ac_indoor_temp"], axis=0))
    result4 = k[["Cmd"]].join(pd.get_dummies(k["Cmd"]).mul(k["ac_outdoor_temp"], axis=0))

    result1 = result1.replace(r'^\s*$', np.NaN, regex=True)
    result2 = result2.replace(r'^\s*$', np.NaN, regex=True)
    result3 = result3.replace(r'^\s*$', np.NaN, regex=True)
    result4 = result4.replace(r'^\s*$', np.NaN, regex=True)

    result1.drop(columns=['CMD_FCU_FROM_AC','CMD_HEARTBEAT'], inplace = True)
    result2.drop(columns=['CMD_FCU_FROM_AC','CMD_HEARTBEAT'], inplace = True)
    result3.drop(columns=['CMD_ENERGY_THIS_MONTH','CMD_ENERGY_THIS_WEEK','CMD_ENERGY_THIS_YEAR',	'CMD_ENERGY_TODAY'], inplace = True)
    result4.drop(columns=['CMD_ENERGY_THIS_MONTH','CMD_ENERGY_THIS_WEEK','CMD_ENERGY_THIS_YEAR',	'CMD_ENERGY_TODAY'], inplace = True)

    result1 = result1.rename(columns={'CMD_ENERGY_THIS_MONTH':'EM-fcuTime', 'CMD_ENERGY_THIS_WEEK':'EW-fcuTime','CMD_ENERGY_TODAY':'ET-fcuTime','CMD_ENERGY_THIS_YEAR':'EY-fcuTime'})
    result2 = result2.rename(columns={'CMD_ENERGY_THIS_MONTH':'EM-energy', 'CMD_ENERGY_THIS_WEEK':'EW-energy','CMD_ENERGY_TODAY':'ET-energy','CMD_ENERGY_THIS_YEAR':'EY-energy'})
    result3 = result3.rename(columns={'CMD_FCU_FROM_AC':'FCU-ac_indoor_temp','CMD_HEARTBEAT':'HB-ac_indoor_temp'})
    result4 = result4.rename(columns={'CMD_FCU_FROM_AC':'FCU-ac_outdoor_temp', 'CMD_HEARTBEAT':'HB-ac_outdoor_temp'})

    result1 = result1.join(result2[['EM-energy','EW-energy','EY-energy','ET-energy']]).join(result3[['FCU-ac_indoor_temp','HB-ac_indoor_temp']]).join(result4[['FCU-ac_outdoor_temp','HB-ac_outdoor_temp']])

    k = k.join(result1[['EM-fcuTime','EW-fcuTime','EY-fcuTime','ET-fcuTime','EM-energy','EW-energy','EY-energy','ET-energy',
                        'FCU-ac_indoor_temp','HB-ac_indoor_temp','FCU-ac_outdoor_temp','HB-ac_outdoor_temp']])

    k.drop(columns=['fcuTime','energy','ac_outdoor_temp','ac_indoor_temp'], inplace = True)

    k.to_csv('/content/drive/MyDrive/Tai-Sana_exchangeDrive/pr-1_1/Result_TAI/Mar22-Cleaning-0108/' + 'cleaning' + i)


  0%|          | 0/20 [00:00<?, ?it/s]

Len of orginal file: 10306175
Len of k after check SourceID and Cmd: 10306175
Len of k after remove Old ACs: 4857691
Len of k after check timestampt: 4722800
Len of k after check  the ac_outdoor_temp: 4722800
Len of k after check ac_scheduler_status is exist or not: 4680243
Len of k after check ac_uiled_status is exist or not: 4680034
len of k after check CMD: 4680034


  5%|▌         | 1/20 [16:18<5:09:56, 978.78s/it]

Len of orginal file: 10306179
Len of k after check SourceID and Cmd: 10306179
Len of k after remove Old ACs: 4856078
Len of k after check timestampt: 4721316
Len of k after check  the ac_outdoor_temp: 4721316
Len of k after check ac_scheduler_status is exist or not: 4678617
Len of k after check ac_uiled_status is exist or not: 4678397
len of k after check CMD: 4678396


 10%|█         | 2/20 [34:18<5:11:24, 1038.05s/it]

...

Len of orginal file: 10306178
Len of k after check SourceID and Cmd: 10306178
Len of k after remove Old ACs: 4858369
Len of k after check timestampt: 4723332
Len of k after check  the ac_outdoor_temp: 4723332
Len of k after check ac_scheduler_status is exist or not: 4680297
Len of k after check ac_uiled_status is exist or not: 4680070
len of k after check CMD: 4680070


 95%|█████████▌| 19/20 [5:57:35<19:25, 1165.72s/it]

Len of orginal file: 10306175
Len of k after check SourceID and Cmd: 10306175
Len of k after remove Old ACs: 4859201
Len of k after check timestampt: 4724037
Len of k after check  the ac_outdoor_temp: 4724037
Len of k after check ac_scheduler_status is exist or not: 4681056
Len of k after check ac_uiled_status is exist or not: 4680820
len of k after check CMD: 4680820


100%|██████████| 20/20 [6:17:24<00:00, 1132.22s/it]

time: 6h 17min 24s (started: 2022-08-01 15:23:06 +00:00)
